In [1]:
import time
import timeit
import math

import numpy as np
import timeit
import numba as nb
import importlib

In [2]:
_number = int(1e3)
one_expr = r"0.44*exp(-407.0*Tgas**(-1/3)+824.0*Tgas**(-2/3))"
_multi_list = []
for i in range(_number):
    _random_n = np.random.randn()
    _multi_list.append(f"{_random_n:.2f}*exp(-407.0*Tgas**(-1/3)+824.0*Tgas**(-2/3))")
    # _multi_list.append(f"{_random_n:.2f} + Tgas")
_str = ",".join(_multi_list)
multi_expr = "[" + _str + "]"

one_expr_compiled = compile(one_expr,'<string>','eval')
multi_expr_compiled = compile(multi_expr,'<string>','eval')
one_expr_lamb = eval('lambda Tgas: ' + one_expr,{"__builtins__":__builtins__,"exp":math.exp},{})
multi_expr_lamb = eval('lambda Tgas:' + multi_expr,{"__builtins__":__builtins__,"exp":math.exp})

%timeit eval(one_expr,{"__builtins__":__builtins__,},dict(exp=math.exp,Tgas=2000))
%timeit eval(one_expr_compiled,{"__builtins__":__builtins__,"exp":math.exp,"Tgas":2000})
%timeit one_expr_lamb(Tgas=2000)

In [3]:
%timeit eval(multi_expr,{"__builtins__":__builtins__,},dict(exp=math.exp,Tgas=2000))
%timeit eval(multi_expr_compiled,{"__builtins__":__builtins__,"exp":math.exp,"Tgas":2000})
%timeit multi_expr_lamb(Tgas=2000)

13.5 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
440 µs ± 15.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
420 µs ± 25.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [4]:
%cd "C:\Users\GuiErGuiEr\Documents\Code\PlasmaChemistry"

C:\Users\GuiErGuiEr\Documents\Code\PlasmaChemistry


In [5]:
with open(r"_cache/cache_template.py") as f:
    _str = f.readlines()
_str = "".join(_str)
# _list = [f"value[{i}] = {_}" for i,_ in enumerate(_multi_list)]
# _list = [f"va.append({_})" for i,_ in enumerate(_multi_list)]
_list = [f"{_}" for i,_ in enumerate(_multi_list)]
# _multi_expr_str = "\n    ".join(_list)
_multi_expr_str = ",".join(_list)

In [6]:
# _str_to_write = _str.replace('"""__REPLACE__"""', f"a = np.zeros({_number})\n    {_multi_expr_str}")
_str_to_write = _str.replace('"""__REPLACE__"""', f"return np.array([{_multi_expr_str}])")

In [7]:
with open(r"_cache/cache.py", "r+") as f:
    f.seek(0)
    f.truncate()
    f.write(_str_to_write)

In [8]:
start = time.time()
%run _cache/cache.py
end = time.time()

In [11]:
print(f"{-start+end:.1f}")

29.0


In [9]:
import _cache.my_module_1 as m

In [10]:
%timeit m.test(3000)

945 ns ± 9.83 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
